In [10]:
import os
# on se replace à la racine du projet pour utiliser les modules créé dans le projet
os.chdir("../../..")
from data_storage.sql_alchemy_connect.alchemy_cur import AlchemyConn
import pandas as pd



In [11]:
connector = AlchemyConn()
df = pd.read_sql('SELECT * FROM raw_data',con=connector.get_engine())

In [12]:
print(df.head())
print(df.shape)

   id  surface_reelle_bati   prix_m2  nombre_pieces_principales  longitude  \
0   3                 78.0  11643.60                        3.0    2.25112   
1   4                 70.0   8571.43                        3.0    2.25190   
2   5                 77.0  10066.60                        4.0    2.25243   
3   6                 67.0   8208.96                        3.0    2.25287   
4   7                 74.0   9331.08                        3.0    2.25137   

   latitude  nombre_lots  annee  arrondissement  distance_datashop_km  ...  \
0   48.8430            2   2020              16              1.123880  ...   
1   48.8384            4   2020              16              0.152864  ...   
2   48.8377            1   2020              16              0.769220  ...   
3   48.8374            2   2020              16              0.410568  ...   
4   48.8393            2   2020              16              0.269765  ...   

   distance_universite_km  distance_ecole_km  distance_metro_k

In [13]:
y = df['prix_m2']
X = df.drop(['prix_m2'],axis=1)
print(y.head())

0    11643.60
1     8571.43
2    10066.60
3     8208.96
4     9331.08
Name: prix_m2, dtype: float64


In [14]:
print(X.isnull().sum())
print('\n-------\n')
print(y.isnull().sum())

id                           0
surface_reelle_bati          0
nombre_pieces_principales    0
longitude                    0
latitude                     0
nombre_lots                  0
annee                        0
arrondissement               0
distance_datashop_km         0
distance_espace_vert_km      0
distance_college_km          0
distance_universite_km       0
distance_ecole_km            0
distance_metro_km            0
distance_TER_km              0
distance_POI_min_km          0
proche_POI_1km               0
nb_POIs_inf_1km              0
cle_interop_adr_proche       0
distance_batiment_m          0
annee_construction_dpe       0
dtype: int64

-------

0


In [15]:
X.dtypes

id                             int64
surface_reelle_bati          float64
nombre_pieces_principales    float64
longitude                    float64
latitude                     float64
nombre_lots                    int64
annee                          int64
arrondissement                 int64
distance_datashop_km         float64
distance_espace_vert_km      float64
distance_college_km          float64
distance_universite_km       float64
distance_ecole_km            float64
distance_metro_km            float64
distance_TER_km              float64
distance_POI_min_km          float64
proche_POI_1km                 int64
nb_POIs_inf_1km                int64
cle_interop_adr_proche        object
distance_batiment_m          float64
annee_construction_dpe       float64
dtype: object

In [16]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import numpy as np

dbscan = DBSCAN(eps=0.001,min_samples=5,metric='euclidean',algorithm='auto')
sc = StandardScaler()

df_cluster = df[['latitude','longitude']]
df_cluster = sc.fit_transform(df_cluster)

dbscan.fit(df_cluster)

,eps,0.001
,min_samples,5
,metric,'euclidean'
,metric_params,None
,algorithm,'auto'
,leaf_size,30
,p,None
,n_jobs,None


In [17]:
print(set(dbscan.labels_))
print(f"Il y a : {len(set(dbscan.labels_))} clusters")
print(f"noise : {list(dbscan.labels_).count(-1)}")

{np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21), np.int64(22), np.int64(23), np.int64(24), np.int64(25), np.int64(26), np.int64(27), np.int64(28), np.int64(29), np.int64(30), np.int64(31), np.int64(32), np.int64(33), np.int64(34), np.int64(35), np.int64(36), np.int64(37), np.int64(38), np.int64(39), np.int64(40), np.int64(41), np.int64(42), np.int64(43), np.int64(44), np.int64(45), np.int64(46), np.int64(47), np.int64(48), np.int64(49), np.int64(50), np.int64(51), np.int64(52), np.int64(53), np.int64(54), np.int64(55), np.int64(56), np.int64(57), np.int64(58), np.int64(59), np.int64(60), np.int64(61), np.int64(62), np.int64(63), np.int64(64), np.int64(65), np.int64(66), np.int64(67), np.int64(68), np.int64(69), np.int64(70), np.int64(71), n

In [18]:
import plotly.express as px
df_map = pd.DataFrame({
    'latitude': df['latitude'],
    'longitude': df['longitude'],
    'cluster': dbscan.labels_
})
fig = px.scatter_map(
    df_map,
    lat="latitude",
    lon="longitude",
    color="cluster",
    hover_data=["cluster"],
    zoom=11,
    center={"lat": 48.8566, "lon": 2.3522},
    map_style="carto-positron"  
)
fig.show()

In [19]:
df_map['cluster'].value_counts()

cluster
-1       54648
 5947      289
 4988      211
 2477      148
 2400      136
         ...  
 6721        5
 6725        5
 6726        5
 6730        5
 3735        5
Name: count, Length: 9374, dtype: int64